In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Endometriosis"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Endometriosis/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Endometriosis/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Endometriosis/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Endometriosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os
import pandas as pd

# 1. Find the most relevant directory for Endometriosis
subdirectories = os.listdir(tcga_root_dir)
target_trait = trait.lower().replace("_", " ")  # Convert to lowercase for case-insensitive matching

# Search for related terms to Endometriosis
related_terms = ["endometrio", "uterine", "uterus", "endometrial", "ucec"]
matched_dir = None

for subdir in subdirectories:
    subdir_lower = subdir.lower()
    # Check if any related term is in the directory name
    if any(term in subdir_lower for term in related_terms):
        matched_dir = subdir
        print(f"Found potential match: {subdir}")
        # If exact match found, select it
        if "endometrio" in subdir_lower:
            print(f"Selected as best match: {subdir}")
            matched_dir = subdir
            break

# If we found a potential match, use it
if matched_dir:
    print(f"Selected directory: {matched_dir}")
    
    # 2. Get the clinical and genetic data file paths
    cohort_dir = os.path.join(tcga_root_dir, matched_dir)
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical file: {os.path.basename(clinical_file_path)}")
    print(f"Genetic file: {os.path.basename(genetic_file_path)}")
    
    # 3. Load the data files
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
    
    # 4. Print clinical data columns for inspection
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Print basic information about the datasets
    print(f"\nClinical data shape: {clinical_df.shape}")
    print(f"Genetic data shape: {genetic_df.shape}")
    
    # Check if we have both gene and trait data
    is_gene_available = genetic_df.shape[0] > 0
    is_trait_available = clinical_df.shape[0] > 0
    
else:
    print(f"No suitable directory found for {trait}.")
    is_gene_available = False
    is_trait_available = False

# Record the data availability
validate_and_save_cohort_info(
    is_final=False,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Exit if no suitable directory was found
if not matched_dir:
    print("Skipping this trait as no suitable data was found.")


Found potential match: TCGA_Uterine_Carcinosarcoma_(UCS)
Found potential match: TCGA_Endometrioid_Cancer_(UCEC)
Selected as best match: TCGA_Endometrioid_Cancer_(UCEC)
Selected directory: TCGA_Endometrioid_Cancer_(UCEC)
Clinical file: TCGA.UCEC.sampleMap_UCEC_clinicalMatrix
Genetic file: TCGA.UCEC.sampleMap_HiSeqV2_PANCAN.gz



Clinical data columns:
['CDE_ID_3226963', '_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_PANCAN', '_PANCAN_DNAMethyl_UCEC', '_PANCAN_RPPA_PANCAN_K8', '_PANCAN_UNC_RNAseq_PANCAN_K16', '_PANCAN_miRNA_PANCAN', '_PANCAN_mirna_UCEC', '_PANCAN_mutation_PANCAN', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_treatment_completion_success_outcome', 'age_at_initial_pathologic_diagnosis', 'aln_pos_ihc', 'aln_pos_light_micro', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'birth_control_pill_history_usage_category', 'clinical_stage', 'colorectal_cancer', 'days_to_additional_surgery_metastatic_procedure', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_last_known_alive', 'days_to_new_tumor_event_additional_surge

### Step 2: Find Candidate Demographic Features

In [3]:
# Identify candidate columns for age and gender
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth']
candidate_gender_cols = ['gender']

# Load the clinical data file to access these columns
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(os.path.join(tcga_root_dir, "TCGA_Endometrioid_Cancer_(UCEC)"))
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Extract and preview age columns
if candidate_age_cols:
    age_preview_dict = {}
    for col in candidate_age_cols:
        if col in clinical_df.columns:
            age_preview_dict[col] = clinical_df[col].head(5).tolist()
    print("Age columns preview:")
    print(age_preview_dict)

# Extract and preview gender columns
if candidate_gender_cols:
    gender_preview_dict = {}
    for col in candidate_gender_cols:
        if col in clinical_df.columns:
            gender_preview_dict[col] = clinical_df[col].head(5).tolist()
    print("Gender columns preview:")
    print(gender_preview_dict)


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [59.0, 54.0, 69.0, 51.0, 67.0], 'days_to_birth': [nan, -19818.0, -25518.0, -18785.0, -24477.0]}
Gender columns preview:
{'gender': ['FEMALE', 'FEMALE', 'FEMALE', 'FEMALE', 'FEMALE']}


### Step 3: Select Demographic Features

In [4]:
# Examine the age columns
# 'age_at_initial_pathologic_diagnosis' has numeric values and appears to be the most direct
# 'days_to_birth' has some missing values (nan) and would need conversion (negative values)

# Examine the gender column
# There is only one gender column 'gender' with values like 'FEMALE'

# Select appropriate columns
age_col = 'age_at_initial_pathologic_diagnosis'
gender_col = 'gender'

# Print information about chosen columns
print(f"Chosen age column: {age_col}")
print(f"First 5 values from age column: [59.0, 54.0, 69.0, 51.0, 67.0]")

print(f"Chosen gender column: {gender_col}")
print(f"First 5 values from gender column: ['FEMALE', 'FEMALE', 'FEMALE', 'FEMALE', 'FEMALE']")


Chosen age column: age_at_initial_pathologic_diagnosis
First 5 values from age column: [59.0, 54.0, 69.0, 51.0, 67.0]
Chosen gender column: gender
First 5 values from gender column: ['FEMALE', 'FEMALE', 'FEMALE', 'FEMALE', 'FEMALE']


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Create clinical features dataframe with trait (Canavan Disease) using patient IDs
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait, 
    age_col=age_col, 
    gender_col=gender_col
)

# Step 2: Normalize gene symbols in the gene expression data
# The gene symbols in TCGA genetic data are already standardized, but we'll normalize them for consistency
normalized_gene_df = normalize_gene_symbols_in_index(genetic_df)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")
print(f"Gene expression data shape after normalization: {normalized_gene_df.shape}")

# Step 3: Link clinical and genetic data
# Transpose genetic data to have samples as rows and genes as columns
genetic_df_t = normalized_gene_df.T
# Save the clinical data for reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print(f"Clinical data shape: {clinical_features.shape}")

# Verify common indices between clinical and genetic data
clinical_indices = set(clinical_features.index)
genetic_indices = set(genetic_df_t.index)
common_indices = clinical_indices.intersection(genetic_indices)
print(f"Number of samples in clinical data: {len(clinical_indices)}")
print(f"Number of samples in genetic data: {len(genetic_indices)}")
print(f"Number of common samples: {len(common_indices)}")

# Link the data by using the common indices
linked_data = pd.concat([clinical_features.loc[list(common_indices)], genetic_df_t.loc[list(common_indices)]], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# Step 4: Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# Step 5: Determine whether the trait and demographic features are severely biased
trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait=trait)

# Step 6: Conduct final quality validation and save information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=trait_biased,
    df=linked_data,
    note=f"Dataset contains TCGA glioma and brain tumor samples with gene expression and clinical information for {trait}."
)

# Step 7: Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable based on validation criteria. Data not saved.")

print("Preprocessing completed.")

Normalized gene expression data saved to ../../output/preprocess/Endometriosis/gene_data/TCGA.csv
Gene expression data shape after normalization: (19848, 201)
Clinical data saved to ../../output/preprocess/Endometriosis/clinical_data/TCGA.csv
Clinical data shape: (596, 3)
Number of samples in clinical data: 596
Number of samples in genetic data: 201
Number of common samples: 201
Linked data shape: (201, 19851)


Data shape after handling missing values: (201, 19851)
For the feature 'Endometriosis', the least common label is '0' with 24 occurrences. This represents 11.94% of the dataset.
The distribution of the feature 'Endometriosis' in this dataset is fine.

Quartiles for 'Age':
  25%: 58.0
  50% (Median): 65.24598930481284
  75%: 72.0
Min: 33.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 201 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



Linked data saved to ../../output/preprocess/Endometriosis/TCGA.csv
Preprocessing completed.
